In [1]:
import dtext as dt
import tensorflow as tf
import getpass
import ipywidgets as ipw
import os
import json
import shlex
import re
from PIL import Image
import logging
import traceback
import time

#model_path="./models/09.10.2019.on_5M.hd5"
#new_model=tf.keras.models.load_model(model_path)
#batch iterator via XNAT hierarchy.

class ServerParams:
    '''
    Container parameters received from XNAT
    '''
    def __init__(self,server=None, user=None, password=None, project=None,subject=None,experiment=None):
        self.server,self.user,self.password,self.project,self.subject,self.experiment= \
            server,user,password,project,subject,experiment
        self.jsession=''
        self.connected=False

    def __str__(self):
        return "server:{}, user: {}, project: {}, subject: {}, experiment: {}, connected: {}".\
            format(self.server,self.user,self.project,self.subject,self.experiment,self.connected)
        
    def connect(self):
        cmd="curl -o jsession.txt -k -u "+ self.user+":"+self.password+ \
            " "+self.server+"/data/JSESSION"
        os.system(cmd)
        with open("jsession.txt") as f:
            self.jsession=f.read()           
        self.connected=(len(self.jsession)==32)
        return self.connected
    
class XnatIterator:
    def __init__(self,sp):
        self.sp=sp
        self._subjects=[]
        self._experiments=[]
        self._scans=[]
            
    def _curl_cmd_prefix(self):
        return "curl  -k --cookie JSESSIONID=" + self.sp.jsession
    
    def _curl_cmd_path(self,path):
        return shlex.quote(self.sp.server+"/data/archive/projects/"+self.sp.project+path)
    
    def _curl_cmd(self,path):        
        cmd=self._curl_cmd_prefix()+' -o temp_query.json '+self._curl_cmd_path(path)
        !rm -f temp_query.json
        os.system(cmd)
        
    def curl_download_single_file(self,path,dest):
        cmd=self._curl_cmd_prefix()+' -o '+dest+' '+ self.sp.server + path
        #print(cmd)
        os.system(cmd)
        
    def set_project(self,pr):
        self.sp.project=pr
    
    def list_subjects(self):
        self._curl_cmd('/subjects?format=json')
        with open ('temp_query.json') as tq:
            try: 
                df=json.loads(tq.read())
            except:
             #   print ('cannot list subjects')
                return []
        #print(df)
        subjs=sorted(df['ResultSet']['Result'], key=lambda k:k['label'])        
        self._subjects=[f['label'] for f in subjs]
        return self._subjects
    def list_experiments(self,subject):
        self._curl_cmd('/subjects/'+subject+"/experiments?xsiType=xnat:imageSessionData&format=json")        
        with open ('temp_query.json') as tq:
            try: 
                df=json.loads(tq.read())
            except: 
                print ('error listing experiments!')
                return []
        #print(df['ResultSet']['Result'])
        exps=sorted(df['ResultSet']['Result'], key=lambda k:k['date'])
        self._experiments=[f['label'] for f in exps]
        return self._experiments
    
    def list_scans(self,subject,experiment, listDcmFiles=False):
        self._curl_cmd('/subjects/'+ subject +'/experiments/' \
            +experiment + "/scans?columns=ID,frames,type,series_description")
        
        with open ('temp_query.json') as sf:
            try: df=json.loads(sf.read())
            except:
                #print ('cannot list scans')
                return []
        self._scans=sorted(df['ResultSet']['Result'], key=lambda k:k['xnat_imagescandata_id'])
        if listDcmFiles:
            for s in self._scans:
                files=self.list_scan_files(subject,experiment,s['ID'])
                s['files']=files
        return self._scans
    
    def get_dcm_files_for_scans(self,subject,experiment,scans):
        for s in scans:
            files=self.list_scan_files(subject,experiment,s['ID'])
            s['files']=files
        
    def list_scan_files(self,subject,experiment,scan):
        self._curl_cmd('/subjects/'+ subject +'/experiments/' \
            +experiment + '/scans/'+scan+'/resources/DICOM/files')
        with open ('temp_query.json') as sf:
            try: df=json.loads(sf.read())
            except:
                #print ('cannot list scans')
                return []
        lst=sorted(df['ResultSet']['Result'], key=lambda k:k['Name'])
        return [ f['URI'] for f in lst ]        
    """
    list all scans in project, filtered by subject prefix. 
    Display progres in output textarea.
    Save output in speficified json file.
    """
    def list_scans_all(self,subjects,subject_prefix,json_out_file,output):
        scans=[]
        for s in subjects:
            if not s.startswith(subject_prefix): continue
            experiments=self.list_experiments(s)
            for e in experiments:
                if output: output.value='{}/{}'.format(s,e)
                scans.append(self.list_scans(s,e))
            with open(json_out_file, 'w') as fp:
                json.dump(scans, fp)
        return scans


/home/NRG/mmilch01/lib/tensorflow-build/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/NRG/mmilch01/lib/tensorflow-build/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/NRG/mmilch01/lib/tensorflow-build/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [3]:
class Selector:
    '''
    GUI to select a particular experiment to process.
    '''       
    
    def on_connect(self,b):
        self.lbl1.value='status: connecting...'
        self.sp.server,self.sp.user,self.sp.password=self.text1.value,self.text2.value,self.text3.value
        if self.sp.connect():                    
            self.lbl1.value='status: connected'
            self.btn1.description='reconnect'            
            if self._first_time: self.show()
            self._project_list()
            self._first_time=False
            #self.btn1.disabled=True
        else:
            self.lbl1.value='status: connection failed ( possibly incorrect combination of user and password )'
                
    def show_login_form(self):                
        st={'description_width':'initial'}
        layout=ipw.Layout(margin='0 100pt 0 0')
        layout1=ipw.Layout(justify_content='center')
        
        #st={}
        self.text1=ipw.Text(value='https://xnat-dev-mga1.nrg.wustl.edu', description='XNAT server:', 
                            layout={'width':'200pt'}, style=st, disabled=False)
        self.text2=ipw.Text(value='admin',description='user:',
                            placeholder='wwwww', disabled=False, style=st, layout={'width':'120pt'})
        self.text3=ipw.Password(value='admin',description='password:',
                                placeholder='wwwww', disabled=False, style=st, layout={'width':'120pt'})
        self.lbl1=ipw.Label('status: not connected', layout={'width':'120pt'}, style=st) #layout={'width':'240px','justify-content':'center'}
        lbl2=ipw.Label('',layout={'width':'120pt'},style=st)
        self.btn1=ipw.Button(description="connect",style={},layout={'width':'200pt'})
        self.btn1.on_click(self.on_connect)
        vb1=ipw.HBox([self.text1,self.text2,self.text3])
        vb2=ipw.HBox([self.btn1,lbl2,self.lbl1])
        self._login_box=ipw.VBox([vb1,vb2])
        display(self._login_box)
        
        #display(self.text1); display(self.text2); display(self.text3); display(self.lbl1); display(self.btn1)    
    
    def _query_prefix(self):
        return "curl -o mga_temp_query.json -k --cookie JSESSIONID=" + \
            self.sp.jsession + " " + self.sp.server+"/data/archive/projects/"
        
    def show(self):
        #self.show_login_form()
        display(self._box); display(self._status); display(self.show_scans())
        self.set_enable(True)
    
    def _project_list(self):
        if not self.sp.connected: self._scans_status('not connected'); return
        cmd=self._query_prefix()+'?format=json'
        self._status.value='status: listing projects...'
        #print('project_list:'+cmd)
        os.system(cmd)
        with open ('mga_temp_query.json') as tq:
            #try: 
            df=json.loads(tq.read())
            #except: 
            #   print ('cannot read mga_temp_query.json')
            #    return
        projs=sorted(df['ResultSet']['Result'], key=lambda k:k['ID'])
        
        self._projects=[f['ID'] for f in projs]
        self._project_selector.options=self._projects
        self._status.value='status: ready'
              
    def _subject_list(self):
        if not self.sp.connected: self._scans_status('not connected'); return
        if self._project_selector.value is None: return
        cmd=self._query_prefix()+self._project_selector.value+'/subjects?format=json'
        #print('subject_list:'+cmd)
        #print(cmd)
        self._status.value='status: listing subjects...'
        os.system(cmd)
        with open ('mga_temp_query.json') as tq:
            try: 
                df=json.loads(tq.read())
            except: return
        #print(df)
        subjs=sorted(df['ResultSet']['Result'], key=lambda k:k['label'])        
        self._subjects=[f['label'] for f in subjs]
        self._subject_selector.options=self._subjects
        self._status.value='status: ready'
 #       if (len(self.subjects)>0): self.subject_selector.value=self.subjects[0]
    
    def _experiment_list(self):
        if not self.sp.connected: self._scans_status('not connected'); return
        if self._project_selector.value is None or self._subject_selector.value is None: return
        #?xsiType=xnat:mrSessionData
        cmd="curl -o mga_temp_query.json -k -u " + self.sp.user+":"+self.sp.password+" " \
            +shlex.quote(self.sp.server+"/data/archive/projects/"+self._project_selector.value+ \
            '/subjects/'+self._subject_selector.value+"/experiments?xsiType=xnat:mrSessionData&format=json")
        self._status.value='status: listing experiments...'
        #print("experiment_list:"+cmd)
        !rm mga_temp_query.json
        df=[]
        os.system(cmd)
        #!{cmd}
        with open ('mga_temp_query.json') as tq:
            try: 
                df=json.loads(tq.read())
            except: 
                print ('error listing experiments!')
                return
        #print(df['ResultSet']['Result'])
        #return
        exps=sorted(df['ResultSet']['Result'], key=lambda k:k['date'])
        self._experiments=[f['label'] for f in exps]
        self._experiment_selector.options=self._experiments
        self._status.value='status: ready'
#        if (self.render_scans==True): 
 #           self._render_scans()
  #      if (len(self.experiments)>0): self.experiment_selector.value=self.experiments[0]

    def _scan_list(self):
        if not self.sp.connected: self._scans_status('not connected'); return
        if self._project_selector.value is None \
            or self._subject_selector.value is None \
            or self._experiment_selector.value is None: return
        cmd=self._query_prefix()+self._project_selector.value+'/subjects/'+ \
                self._subject_selector.value+'/experiments/'+self._experiment_selector.value + \
                "/scans?columns=ID,frames,type,series_description"
        #print('scan list:'+cmd)
        #print(cmd)
        self._status.value='status: listing scans...'
        os.system(cmd)
        with open ('mga_temp_query.json') as sf:
            try: df=json.loads(sf.read())
            except: return
        self.scans=sorted(df['ResultSet']['Result'], key=lambda k:k['xnat_imagescandata_id'])
        self._status.value='status: ready'
        
    def _render_scans_header(self):
        style={'description_width':'initial'}
        row=ipw.HBox([
            ipw.Label(value='Scan',style=style,layout={'width':'70px'}),
            ipw.Label(value='Description',style=style,layout={'width':'200px'}),
            ipw.Label(value='Frames',style=style,layout={'width':'60px'}),
            ipw.Label(value='Text detected',style=style,layout={'width':'80px'}),
            ipw.Label(value='Preview',style=style,layout={'width':'50px'}),
            ipw.Label(value='PHI',style=style,layout={'width':'50px'})
            #ipw.Label(value='HOF ID',style=style,layout={'width':'60px'})
        ])
        return row
    
    def _hide_scans(self):
        self.scans=[]
        self.scanbox.children=[]
    
    def _render_scan_row(self,scan):        
        style={'description_width':'initial'}
        descr=scan['series_description']
        descr=re.sub('[^0-9a-zA-Z ]+',' ',descr)
        s=!slist qd {descr}
        hof_info=s.n.split()
        if (len(hof_info)==4):
            hof_id=hof_info[1]
            checked=True
        else:
            hof_id='select'
            checked=False
        #print(scan)
        if 'dtext_res' in scan:
            has_text=(scan['dtext_res']['text_present']==1)
            ib=( ('DETECTED' if has_text else 'no'), not has_text, not has_text )
        else:
            ib=('unknown',True,True)
        
        btn=ipw.Button(description='view', tooltip=scan['ID'], disabled=ib[1],layout={'width':'50px'})
        btn.on_click(self.on_dtext_view)
        row=ipw.HBox([
            ipw.Label(value=scan['ID'],style=style,layout={'width':'70px'}),
            ipw.Label(value=descr,style=style,layout={'width':'200px'}),
            ipw.Label(value=scan['frames'],style=style,layout={'width':'60px'}),
            ipw.Label(value=ib[0],style=style,layout={'width':'80px'}),
            btn,
            ipw.Checkbox(value=False, description='', disabled=ib[2], style=style, layout={'width':'50px'})
            #ipw.Dropdown(options=self._hof_ids,value=hof_id,style=style,layout={'width':'200px'})
        ])
        
        #row.children[0].observe(on_scanlist_change,names='value')
        #row.children[3].observe(on_scanlist_change,names='value')
        return row
    
    def _scans_status(self,txt,clear=False):
        if clear: self._output.clear_output()
        with self._output:            
            print(txt)
    
    def reset_viewbox(self):
        self._action_btn.disabled=True
        self._preview_im=ipw.Image(width=1, height=400)
        self._output.clear_output()
        
    def show_scans(self, update_scans=True):
        if (self.render_scans==False): return        
        self._status.value="status: loading scans"
        if update_scans:
            self._scan_list()
            self.reset_viewbox()
            
        #self._scanbox.children=[]
        rows=[self._render_scans_header()]
        for s in self.scans:            
            row=self._render_scan_row(s)
            rows.append(row)
            
        #self.scanrow_box.children=[rows, [self._action_btn]]
        if update_scans and len(rows)>1: 
            self._action_btn.disabled=False
            self._action_btn.description='detect text'
        rows.append(self._action_btn)
        
        #left vertical box.
        self.scanrow_box.children=rows
        
        #right vertical box.
        self._aux_box.children=[self._preview_im,self._output]
        
        #enveloping horizontal box.
        self.scanbox.children=[self.scanrow_box,self._aux_box]
        self._status.value="status: ready"
        return self.scanbox
    
    def show_scans_dtext(self, res):
        if (self.render_scans==False): return
        self._status.value="status: loading scans"
        if len(self.scans)<1:      
            self._scan_list()            
        rres={}
        for r in res:
            ss=r['infile'].split('/')[-1].rstrip('.png')
            #ss=r['infile'].replace('.png','')
            rres[ss]=r            
        for s in self.scans:
            #print(s['ID'])
            if s['ID'] in rres:
                r=rres[s['ID']]
                s['dtext_res']=r
                #print("updating scan:")
                #print(s)
                
        return self.show_scans(False)
    
    def on_dtext_view(self,b):
        if not self.sp.connected: _scans_status('not connected'); return
        #identify target scan.
        d={}
        for ss in self.scans:
            d[ss['ID']]=ss
        s=d[b.tooltip]['dtext_res']
        with open('./xnat_temp/'+s['outfile'], "rb") as f:            
            img = f.read()
        self._preview_im.close()
        self._scans_status('previewing detected text in scan '+b.tooltip,True)
        self._preview_im=ipw.Image(value=img,width=400, height=400,format='png',layout={'width':'400px','height':'400px'})        
        self._aux_box.children=[self._preview_im,self._output]
        self.scanbox.children=[self.scanrow_box, self._aux_box]
        
        #display(self._preview_im)
        #return self.scanbox
        
    def _on_action_btn(self,b):
        if not self.sp.connected: _scans_status('not connected'); return
        self.set_enable(False)
        b.disabled=True
        self._status.value="status: running text detection..."
        with self._output:
            if b.description=='detect text':
                #run text detection
                cwd=os.getcwd()
                try:
                    #model=tf.keras.models.load_model(cwd+'/models/09.10.2019.on_5M.hd5')
                    print('ETA to process all scans: {} seconds'.format(len(self.scans)*4))
                    t0=time.time()
                    print('retrieving scan filenames...')                    
                    self.xi.sp.project=self.sp.project                
                    self.xi.get_dcm_files_for_scans(self.sp.subject,self.sp.experiment,self.scans)
                    os.system('mkdir -p ./xnat_temp; rm -rf ./xnat_temp/*'); os.chdir('./xnat_temp/')
                    print('loading scan files...')
                    for s in self.scans:                    
                        sid=s['ID']    
                        if len(s['files'])>0:
                            self.xi.curl_download_single_file(s['files'][0],sid+'.dcm')
                            os.system("dcmj2pnm +G +Wn +on "+sid+".dcm "+ sid+".png")
                    os.system('rm *.dcm')

                    print('detecting text...')
                    res=dt.run_detection(self._tf_model,0.99,'./','./out',True)
                    os.chdir(cwd)
                    b.description='Report PHI status'
                    b.disabled=False
                    self.show_scans_dtext(res)
                    self.set_enable(True)
                    print("Time spent: {} seconds".format(time.time()-t0))
                    self._status.value="status: ready"
                except Exception as e:
                    logging.error(traceback.format_exc())
                    os.chdir(cwd)                    
                    #b.disabled=True
                    self.set_enable(True)
                    self._status.value="status: exception"
            else:
                self._scans_status('Placeholder for reporting PHI')
                b.disabled=False
                self._status.value="status: ready"
                self.set_enable(True)
    
    def _on_project_changed(self,p):
        self.set_enable(False)
        self._hide_scans()
        self.sp.project=self._project_selector.value
        self._subject_list()
        temp=self.render_scans; self.render_scans=False
        #self._experiment_list()
        self.render_scans=temp
        self.set_enable(True)
        
    def _on_subject_changed(self,p):        
        self.set_enable(False)
        self._hide_scans()        
        self.sp.subject=self._subject_selector.value
#        temp=self.render_scans; self.render_scans=False
        self._experiment_list()
#        self.render_scans=temp
        self.set_enable(True)
        
    def _on_experiment_changed(self,p):
        self.set_enable(False)
        self._hide_scans()
        self.sp.experiment=self._experiment_selector.value
        self.show_scans()
        self.set_enable(True)        
    
    def set_enable(self,status):
        for c in self.scanbox.children:
            c.disabled=not status
        for c in self._box.children:
            c.disabled=not status
    
    def __init__(self, server_params):
        self.sp=server_params
        self.xi=XnatIterator(server_params)
        style={'description_width':'initial'}
        #style={}
        #layout=Layout()
        #self._project_selector=ipw.Dropdown(description='project:',style=style)
        self._project_selector=ipw.Dropdown(description='project:',style=style,layout={'width':'200px'})
        #print(self._project_selector.style.keys)
        self._subject_selector=ipw.Dropdown(description='subject:',style=style,layout={'width':'300px'})
        #self._subject_selector=ipw.Dropdown(description='subject:',style=style)
        #self._experiment_selector=ipw.Dropdown(description='experiment:',style=style)        
        self._experiment_selector=ipw.Dropdown(description='experiment:',style=style,layout={'width':'300px'})        
        self._first_time=True
                
        
        self._box=ipw.HBox([
            self._project_selector,
            ipw.Label(layout={'width':'10px'}),
            self._subject_selector,
            ipw.Label(layout={'width':'10px'}),
            self._experiment_selector
        ])
        self._status=ipw.Label("status: ready",layout={'width':'300px'})
        
        self.render_scans=True
        self.scans=[]
        self._output=ipw.Output(value='not connected',layout={'width':'500px','height':'400px','overflow_y':'auto'})#'overflow_y':'auto'
        self._preview_im=ipw.Image(width=1, height=400,layout={'width':'400px','height':'400px'})
        
        self._action_btn=ipw.Button(description='Detect text',disabled=True,layout={'width':'200px'})
        self._action_btn.on_click(self._on_action_btn)
        self.scanbox=ipw.HBox([])
        self.scanrow_box=ipw.VBox([])
        self._aux_box=ipw.VBox([])
        self._aux_box.children=[self._preview_im,self._output]
        self.scanbox.children=[self.scanrow_box,self._aux_box]
        
        self._tf_model=tf.keras.models.load_model(os.getcwd()+'/models/09.10.2019.on_5M.hd5')
        
        self._projects=[];self._subjects=[]; self._experiments=[]
        self._project_list()
        self._on_project_changed(self._project_selector)
        self._project_selector.observe(self._on_project_changed,names='value')
        self._subject_selector.observe(self._on_subject_changed,names='value')
        self._experiment_selector.observe(self._on_experiment_changed,names='value')
        
        



In [4]:
#1. conection form.
selector=Selector(ServerParams())
selector.show_login_form()
#sp=ServerParams()
#sp.show_login_form()



Label(value='status: ready', layout=Layout(width='300px'))